# Завдання 2–3

$\begin{cases}
x_1 + x_2 + 2x_3 + 3x_4 = 1 \\
3x_1 - x_2 - x_3 - 2x_4 = -4 \\
2x_1 - 3x_2 - x_3 - x_4 = -6 \\
x_1 + 2x_2 + 3x_3 - x_4 = -4 \\
\end{cases}$

In [1]:
import numpy as np
import time
import cProfile

A = np.array([
    [1, 1, 2, 3],
    [3, -1, -1, -2],
    [2, -3, -1, -1],
    [1, 2, 3, -1]
])
b = np.array([1, -4, -6, -4])

# Спосіб b: Крамер через цикли
def cramer_solve(A, b):
    n = len(b)
    det_A = np.linalg.det(A)
    if det_A == 0:
        return None
    solutions = []
    for i in range(n):
        A_i = A.copy()
        A_i[:, i] = b
        det_Ai = np.linalg.det(A_i)
        solutions.append(det_Ai / det_A)
    return np.array(solutions)

start_loop = time.time()
x_loop = cramer_solve(A, b)
end_loop = time.time()
cProfile.run('cramer_solve(A, b)')


print("Крамер:", x_loop)
print("Час Крамер:", end_loop - start_loop)

         64 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 1650935369.py:14(cramer_solve)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 linalg.py:140(isComplexType)
        5    0.000    0.000    0.000    0.000 linalg.py:159(_commonType)
        5    0.000    0.000    0.000    0.000 linalg.py:203(_assert_stacked_2d)
        5    0.000    0.000    0.000    0.000 linalg.py:209(_assert_stacked_square)
        5    0.000    0.000    0.000    0.000 linalg.py:2126(det)
        5    0.000    0.000    0.000    0.000 linalg.py:488(_unary_dispatcher)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
       10    0.000    0.000    0.000    0.000 {built-in method builtins.issubclass}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    

In [3]:
# Спосіб а: NumPy
cProfile.run('np.linalg.solve(A, b)')
start_np = time.time()
x_np = np.linalg.solve(A, b)
end_np = time.time()
print("NumPy:", x_np)
print("Час NumPy:", end_np - start_np)
print("Рівні:", np.allclose(x_np, x_loop))

         21 function calls in 0.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 linalg.py:130(get_linalg_error_extobj)
        2    0.000    0.000    0.000    0.000 linalg.py:135(_makearray)
        1    0.000    0.000    0.000    0.000 linalg.py:140(isComplexType)
        1    0.000    0.000    0.000    0.000 linalg.py:159(_commonType)
        1    0.000    0.000    0.000    0.000 linalg.py:203(_assert_stacked_2d)
        1    0.000    0.000    0.000    0.000 linalg.py:209(_assert_stacked_square)
        1    0.000    0.000    0.000    0.000 linalg.py:325(_solve_dispatcher)
        1    0.002    0.002    0.002    0.002 linalg.py:329(solve)
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
        3   

In [5]:
# Матричне множення
print("Перевірка матричним множенням:", np.allclose(np.dot(A, x_np), b))

# Обернена матриця
A_inv = np.linalg.inv(A)
x_inv = A_inv.dot(b)
print("Обернена матриця:", x_inv)
print("Рівні:", np.allclose(x_np, x_inv))


Перевірка матричним множенням: True
Обернена матриця: [-0.55555556  1.88888889 -2.          1.22222222]
Рівні: True


$$
2(A + B)(2B - A), \quad \text{де} \quad
A = \begin{pmatrix}
2 & 3 & -1 \\
4 & 5 & 2 \\
-1 & 0 & 7
\end{pmatrix}, \quad
B = \begin{pmatrix}
-1 & 0 & 5 \\
0 & 1 & 3 \\
2 & -2 & 4
\end{pmatrix}
$$

In [7]:
A = np.array([[2, 3, -1],
              [4, 5, 2],
              [-1, 0, 7]])
B = np.array([[-1, 0, 5],
              [0, 1, 3],
              [2, -2, 4]])

start_np = time.time()
result_np = 2 * np.dot(A + B, 2 * B - A)
end_np = time.time()
print("NumPy результат:\n", result_np)


NumPy результат:
 [[   8  -56   54]
 [ -30 -100  146]
 [ 118  -82   28]]


In [9]:
# Циклічне обчислення
def mat_mult(A, B):
    result = [[0]*len(B[0]) for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(A[0])):
                result[i][j] += A[i][k] * B[k][j]
    return result

A_plus_B = [[A[i][j] + B[i][j] for j in range(3)] for i in range(3)]
twoB_minus_A = [[2*B[i][j] - A[i][j] for j in range(3)] for i in range(3)]

start_loop = time.time()
mult = mat_mult(A_plus_B, twoB_minus_A)
result_loop = [[2 * el for el in row] for row in mult]
end_loop = time.time()

print("Цикли результат:\n", np.array(result_loop))
print("Рівні:", np.allclose(result_np, result_loop))
print("Час NumPy:", end_np - start_np)
print("Час циклів:", end_loop - start_loop)


Цикли результат:
 [[   8  -56   54]
 [ -30 -100  146]
 [ 118  -82   28]]
Рівні: True
Час NumPy: 0.0
Час циклів: 0.0


## Висновки
- NumPy виконує обчислення значно швидше, особливо при великих розмірах матриць.
- Ітеративний підхід надає гнучкість, але потребує більше часу.
- Всі методи дають однаковий результат, що підтверджується np.allclose().

Завдання 1, 6, 7, 9,
10, 11, 15,
17, 18 
- Вправа 1. Вводиться число n. Вивести масив розміру n×n, в 
якому по діагоналі йдуть числа від 1 до n, а інші числа дорівнюють 0
- Вправа 6. З клавіатури вводиться масив. Замінити всі ненульові
елементи на –1.
- Вправа 7. З клавіатури вводиться масив. Замінити всі нульові
елементи на –1. 
- Вправа 9. Вводиться число n. Створити масив значень від n до 0
- Вправа 10. Вводиться числа n i m. Створити масив випадкових
значень n×m і обчислити мінімальне, максимальне значення, середнє та
середньо квадратичне відхилення, округлене до 3 знаків після коми. 
- Вправа 11. Створити масив одиниць розміром n×n та додати до
нього «рамку», що утворюється з 0. 
- Вправа 15. Заповнити парні стовпчикі матриці розміром n×n
одиницями, а непарні – нулями. 
- Вправа 17. Заповнити вектор значеннями від 0 до n. Замінити
знаки для всіх значень, що менші за n/2 та більші за 3n/4 на
протилежні.
- Вправа 18. Згенерувати вектор з n випадкових чисел в діапазоні
від 0 до 99. Визначити кількість унікальних чисел в послідовності.

- Вправа 1. Вводиться число n. Вивести масив розміру n×n, в 
якому по діагоналі йдуть числа від 1 до n, а інші числа дорівнюють 0

In [16]:
import numpy as np

n = int(input("Введіть n: "))
matrix = np.zeros((n, n), dtype=int)
np.fill_diagonal(matrix, np.arange(1, n+1))
print(matrix)


Введіть n:  5


[[1 0 0 0 0]
 [0 2 0 0 0]
 [0 0 3 0 0]
 [0 0 0 4 0]
 [0 0 0 0 5]]


- Вправа 6. З клавіатури вводиться масив. Замінити всі ненульові
елементи на –1.

In [18]:
arr = np.array(list(map(int, input("Введіть елементи через пробіл: ").split())))
arr[arr != 0] = -1
print(arr)

Введіть елементи через пробіл:  10 1021 121 0 1 -1 21 32 12 3


[-1 -1 -1  0 -1 -1 -1 -1 -1 -1]


- Вправа 7. З клавіатури вводиться масив. Замінити всі нульові
елементи на –1. 

In [20]:
arr = np.array(list(map(int, input("Введіть елементи через пробіл: ").split())))
arr[arr == 0] = -1
print(arr)

Введіть елементи через пробіл:  1 2 31 4 4 1 -1 0 12 33 12 31 33


[ 1  2 31  4  4  1 -1 -1 12 33 12 31 33]


- Вправа 9. Вводиться число n. Створити масив значень від n до 0

In [22]:
n = int(input("Введіть n: "))
arr = np.arange(n, -1, -1)
print(arr)

Введіть n:  12


[12 11 10  9  8  7  6  5  4  3  2  1  0]


- Вправа 10. Вводиться числа n i m. Створити масив випадкових
значень n×m і обчислити мінімальне, максимальне значення, середнє та
середньо квадратичне відхилення, округлене до 3 знаків після коми. 

In [24]:
n, m = map(int, input("Введіть n і m: ").split())
arr = np.random.randint(0, 100, size=(n, m))

mean = np.mean(arr)
std_dev = np.std(arr)
minimum = np.min(arr)
maximum = np.max(arr)

print("Масив:\n", arr)
print(f"Мінімальне: {minimum}")
print(f"Максимальне: {maximum}")
print(f"Середнє: {round(mean, 3)}")
print(f"СКВ: {round(std_dev, 3)}")


Введіть n і m:  12 10


Масив:
 [[94 43 88 18 73  8 60 81 35 21]
 [51  5 14 16 87 69 86 33 32 72]
 [99 67 18 78 94 59 56 61 81 84]
 [79 34 59 82 57 89 46  0 51 59]
 [99  6 42  2 55 15 33 59 89 66]
 [83 75 98 56 64 89 68 64 68 39]
 [ 2 70 44 16 21 50 88  0 69 43]
 [10 79 50 94 89 22 54 24  1 59]
 [24 57  8 18 66 50 17 88 72 55]
 [72 47 93 68 41 91 67 77 27 69]
 [ 3 47 88 62 53 52 55 81  3 52]
 [69 36 16 97 42 84 89 38 58 24]]
Мінімальне: 0
Максимальне: 99
Середнє: 53.75
СКВ: 28.054


- Вправа 11. Створити масив одиниць розміром n×n та додати до
нього «рамку», що утворюється з 0. 

In [32]:
n = int(input("Введіть n: "))
inner = np.ones((n, n), dtype=int)
framed = np.pad(inner, pad_width=1, mode='constant', constant_values=0)
print(framed)

Введіть n:  6


[[0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 0]
 [0 0 0 0 0 0 0 0]]


- Вправа 15. Заповнити парні стовпчикі матриці розміром n×n
одиницями, а непарні – нулями. 

In [34]:
n = int(input("Введіть n: "))
matrix = np.zeros((n, n), dtype=int)
matrix[:, ::2] = 1  # 0, 2, 4... (парні індекси)
print(matrix)

Введіть n:  6


[[1 0 1 0 1 0]
 [1 0 1 0 1 0]
 [1 0 1 0 1 0]
 [1 0 1 0 1 0]
 [1 0 1 0 1 0]
 [1 0 1 0 1 0]]


- Вправа 17. Заповнити вектор значеннями від 0 до n. Замінити
знаки для всіх значень, що менші за n/2 та більші за 3n/4 на
протилежні.

In [36]:
n = int(input("Введіть n: "))
arr = np.arange(0, n+1)
arr[(arr < n/2) | (arr > 3*n/4)] *= -1
print(arr)

Введіть n:  54


[  0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14 -15 -16 -17
 -18 -19 -20 -21 -22 -23 -24 -25 -26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40 -41 -42 -43 -44 -45 -46 -47 -48 -49 -50 -51 -52 -53
 -54]


- Вправа 18. Згенерувати вектор з n випадкових чисел в діапазоні
від 0 до 99. Визначити кількість унікальних чисел в послідовності.

In [40]:
n = int(input("Введіть n: "))
arr = np.random.randint(0, 100, size=n)
unique_count = len(np.unique(arr))
print("Вектор:", arr)
print("Кількість унікальних:", unique_count)

Введіть n:  60


Вектор: [29 27 13 85 35 23 94 24 39 35 69 30 35 82 49 44 48 99 55 10 53 29 83 59
 59 56 55 21 29  3 65 51 17 38 80 87 86 18 36 79 47 77 81 52 28 97 41 61
 59 75 26 79 83 19 64 37 52 17 97 32]
Кількість унікальних: 48
